In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import gmean
import openpyxl

def load_pairwise_matrices(file_path):
    """
    Load pairwise comparison matrices from Excel file with multiple expert sheets
    
    Args:
        file_path (str): Path to the Excel file
        
    Returns:
        tuple: (criteria_names, expert_matrices_dict)
    """
    # Load the Excel file
    wb = openpyxl.load_workbook(file_path)
    
    # Get all sheet names (should be Expert_1, Expert_2, etc.)
    expert_sheets = [sheet for sheet in wb.sheetnames if sheet.startswith('Expert_')]
    
    # Read the first sheet to get criteria names
    first_sheet = wb[expert_sheets[0]]
    
    # Extract criteria names from the first row (starting from column B)
    criteria = []
    for cell in first_sheet[1][1:]:  # Skip the first cell (A1)
        if cell.value is not None:
            criteria.append(cell.value)
        else:
            break
    
    n_criteria = len(criteria)
    expert_matrices = {}
    
    # Read matrices from each expert sheet
    for sheet_name in expert_sheets:
        sheet = wb[sheet_name]
        matrix = []
        
        # Extract the matrix values (starting from row 2, column B)
        for i in range(2, 2 + n_criteria):  # Rows 2 to n_criteria+1
            row = []
            for j in range(2, 2 + n_criteria):  # Columns B to n_criteria+1
                cell_value = sheet.cell(row=i, column=j).value
                if cell_value is not None:
                    row.append(float(cell_value))
                else:
                    row.append(1.0)  # Default diagonal value
            matrix.append(row)
        
        expert_matrices[sheet_name] = np.array(matrix)
    
    wb.close()
    return criteria, expert_matrices

def calculate_ahp_weights(matrix):
    """
    Calculate AHP weights using the geometric mean method (exactly as in your code)
    
    Args:
        matrix (numpy.ndarray): Pairwise comparison matrix
        
    Returns:
        numpy.ndarray: Normalized weights
    """
    # Calculate geometric mean of each row
    n = matrix.shape[0]
    geometric_means = []
    
    for i in range(n):
        # Calculate geometric mean of row i
        row_product = 1.0
        for j in range(n):
            row_product *= matrix[i, j]
        geometric_means.append(row_product ** (1.0 / n))
    
    # Normalize to get weights
    geometric_means = np.array(geometric_means)
    weights = geometric_means / np.sum(geometric_means)
    
    return weights

def calculate_consistency_ratio(matrix, weights):
    """
    Calculate the consistency ratio exactly as in your code
    
    Args:
        matrix (numpy.ndarray): Pairwise comparison matrix
        weights (numpy.ndarray): Priority weights
        
    Returns:
        float: Consistency ratio
    """
    n = matrix.shape[0]
    
    # Calculate lambda_max
    weighted_sum = np.dot(matrix, weights)
    lambda_max = np.sum(weighted_sum / weights) / n
    
    # Calculate consistency index
    ci = (lambda_max - n) / (n - 1)
    
    # Random consistency index for different matrix sizes (your exact values)
    ri_values = {1: 0, 2: 0, 3: 0.52, 4: 0.89, 5: 1.11, 6: 1.25, 7: 1.35, 8: 1.40, 
                 9: 1.45, 10: 1.49, 11: 1.52, 12: 1.54, 13: 1.56, 14: 1.58, 15: 1.59}
    
    ri = ri_values.get(n, 1.59)  # Use 1.59 for matrices larger than 15x15
    
    # Calculate consistency ratio
    cr = ci / ri if ri > 0 else 0
    
    return cr

def kendall_w(rankings):
    """Calculate Kendall's coefficient of concordance"""
    rankings = np.array(rankings)
    m, n = rankings.shape  # m judges, n items
    
    # Convert weights to rankings (1 = highest weight)
    ranked_data = np.zeros_like(rankings)
    for i in range(m):
        ranked_data[i] = n + 1 - np.argsort(np.argsort(rankings[i]))
    
    rank_sums = np.sum(ranked_data, axis=0)
    mean_rank_sum = np.mean(rank_sums)
    s = np.sum((rank_sums - mean_rank_sum)**2)
    
    w = 12 * s / (m**2 * (n**3 - n))
    return w

def calculate_expert_consensus(file_path):
    """
    Calculate expert-derived criteria weights and consensus rankings
    Matches your exact calculation methodology
    
    Args:
        file_path (str): Path to the Excel file with expert matrices
        
    Returns:
        pandas.DataFrame: Results table with weights, rankings, and statistics
    """
    # Load the matrices
    criteria, expert_matrices = load_pairwise_matrices(file_path)
    
    # Calculate weights for each expert
    expert_weights = {}
    expert_consistency = {}
    
    for expert_name, matrix in expert_matrices.items():
        weights = calculate_ahp_weights(matrix)
        cr = calculate_consistency_ratio(matrix, weights)
        
        expert_weights[expert_name] = weights
        expert_consistency[expert_name] = cr
    
    # Create DataFrame with all expert weights
    weights_df = pd.DataFrame(expert_weights, index=criteria)
    
    # Calculate geometric mean across experts for each criterion (your method)
    geometric_mean_weights = []
    for i in range(len(criteria)):
        criterion_weights = [expert_weights[expert][i] for expert in expert_weights.keys()]
        geom_mean = gmean(criterion_weights)
        geometric_mean_weights.append(geom_mean)
    
    # Normalize geometric mean weights
    geometric_mean_weights = np.array(geometric_mean_weights)
    geometric_mean_weights = geometric_mean_weights / np.sum(geometric_mean_weights)
    
    # Calculate standard deviation and coefficient of variation (your method)
    std_devs = []
    cvs = []
    
    for i in range(len(criteria)):
        criterion_weights = [expert_weights[expert][i] for expert in expert_weights.keys()]
        std_dev = np.std(criterion_weights, ddof=1)  # Sample standard deviation
        cv = std_dev / np.mean(criterion_weights)  # CV based on arithmetic mean
        
        std_devs.append(std_dev)
        cvs.append(cv)
    
    # Calculate Kendall's W for inter-expert agreement
    weights_array = np.array([expert_weights[expert] for expert in expert_weights.keys()])
    kendall_w_value = kendall_w(weights_array)
    
    # Create results DataFrame
    results_df = pd.DataFrame({
        'Criterion': criteria,
        'Geometric Mean Weight': geometric_mean_weights,
        'Std Dev': std_devs,
        'CV': cvs
    })
    
    # Sort by geometric mean weight (descending) and add rank
    results_df = results_df.sort_values('Geometric Mean Weight', ascending=False)
    results_df['Rank'] = range(1, len(results_df) + 1)
    
    # Reorder columns to match Table 2 format
    results_df = results_df[['Rank', 'Criterion', 'Geometric Mean Weight', 'Std Dev', 'CV']]
    
    # Print expert panel characteristics and consistency performance
    n_experts = len(expert_matrices)
    consistency_ratios = list(expert_consistency.values())
    mean_cr = np.mean(consistency_ratios)
    std_cr = np.std(consistency_ratios, ddof=1)
    acceptable_consistency = sum(1 for cr in consistency_ratios if cr < 0.1)
    
    print("Expert Panel Characteristics and Consistency Performance")
    print("=" * 60)
    print(f"Panel Size: {n_experts} experts")
    print(f"Mean Consistency Ratio (CR): {mean_cr:.4f} (SD = {std_cr:.4f})")
    print(f"Acceptable Consistency (CR < 10%): {acceptable_consistency}/{n_experts} ({100*acceptable_consistency/n_experts:.0f}%)")
    print(f"Kendall's W (Inter-expert agreement): {kendall_w_value:.3f}")
    
    # Print individual consistency ratios
    print(f"\nConsistency Ratios by Expert:")
    for expert, cr in expert_consistency.items():
        print(f"{expert}: {cr:.4f}")
    
    return results_df, mean_cr, std_cr, kendall_w_value

# Main execution
if __name__ == "__main__":
    # Load and analyze the data
    file_path = "data/simple_matrices.xlsx"  # Update with your file path
    
    try:
        results, mean_cr, std_cr, kendall_w_value = calculate_expert_consensus(file_path)
        
        # Display the final results table (matching your format)
        print("\n" + "="*80)
        print("Table 2. Expert-Derived Criteria Weights and Consensus Rankings")
        print("="*80)
        
        # Format the results for display exactly as you did
        print(f"{'Rank':<6} {'Criterion':<35} {'Geometric Mean Weight':<20} {'Std Dev':<12} {'CV':<8}")
        print("-" * 80)
        
        for _, row in results.iterrows():
            print(f"{row['Rank']:<6} {row['Criterion']:<35} {row['Geometric Mean Weight']:<20.6f} "
                  f"{row['Std Dev']:<12.6f} {row['CV']:<8.4f}")
        
        # Save results to CSV
        results.to_csv("expert_consensus_results.csv", index=False)
        print(f"\nResults saved to 'expert_consensus_results.csv'")
        
    except Exception as e:
        print(f"Error: {e}")
        print("Please make sure the Excel file exists and has the correct format.")